# Deep Learning Final Exam

## Mesyella/2201793281

### 2. Iris Classification with Fastai

Disini saya mengklasifikasikan 3 jenis tipe bunga yaitu iris-setosa, iris-versicolor, dan iris-virginica dengan menggunakan 2 tipe data yaitu data tabular yang terdiri dari 4 variabel dan data gambar.

### A. Code

### Install Torch version

Disini saya menginstall versi torch yang sesuai dengan algoritma yang saya gunakan

In [1]:
!pip install "torch==1.4" "torchvision==0.5.0"

### Install image-tabular library

Library ini merupakan library untuk deep learning yang menggunakan fastai dan pytorch untuk menggabungkan data tabular dan data gambar serta melakukan training model yang telah digabung.
Dokumentasi: https://pypi.org/project/image-tabular/

In [2]:
pip install image_tabular

### Import library

Disini saya import library-library yang dibutuhkan untuk membuat model deep learning yang mengklasifikasi bunga tersebut. Library yang digunakan berasal dari fastai dan juga image_tabular. Selain itu, apabila terdapat gpu maka saya menggunakan gpu, apabila tidak ada maka akan menggunaka cpu untuk runningnya.

In [3]:
from fastai.vision import *
from fastai.tabular import *
from image_tabular.core import *
from image_tabular.dataset import *
from image_tabular.model import *
from image_tabular.metric import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
# use gpu by default if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Prepare Data

Dataset terdiri dari 150 bunga dengan tiga jenis berbeda yaitu Iris-setosa, Iris-versicolor, dan Iris-virginica. Data tabular terdiri dari 4 variabel yaitu sepal_length, sepal_width, petal_length, dan petal_width dan juga data gambar dari bunga tersebut.

Data diambil dari iris-multimodal.csv yang kemudian dibagi menjadi train data dan test data dengan ratio 0.9. Data yang dibagi memiliki jumlah jenis bunga yang seimbang sehingga hasil yang didapatkan tidak condong ke satu jenis bunga.

In [4]:
train_test_ratio = 0.9
data = pd.read_csv('iris_multimodal.csv')
data_setosa = data[data['species'] == 'Iris-setosa']
data_color = data[data['species'] == 'Iris-versicolor']
data_virgin = data[data['species'] == 'Iris-virginica']

# Train-test-split
df_setosa_full_train, df_setosa_test = train_test_split(data_setosa, train_size = train_test_ratio, random_state = 1)
df_color_full_train, df_color_test = train_test_split(data_color, train_size = train_test_ratio, random_state = 1)
df_virgin_full_train, df_virgin_test = train_test_split(data_virgin, train_size = train_test_ratio, random_state = 1)
df_train = pd.concat([df_setosa_full_train, df_color_full_train, df_virgin_full_train], ignore_index=True, sort=False)
df_test = pd.concat([df_setosa_test, df_color_test, df_virgin_test], ignore_index=True, sort=False)

# Encode species
le = LabelEncoder()  
df_train['species']= le.fit_transform(df_train['species']) 
df_test['species']= le.fit_transform(df_test['species']) 

Kemudian menggunakan fastai untuk mengambil data gambar dan data tabular lalu mengubah menjadi fastai labellist.

In [5]:
tfms = get_transforms(flip_vert=True)
size = 128
val_idx = get_valid_index(df_train)
# load image data using train_df and prepare fastai LabelLists
image_data = (ImageList.from_df(df_train, path='.', cols="image",
                               folder="images",)
              .split_by_idx(val_idx)
              .label_from_df(cols="species")
              .transform(tfms, size=size))
test_image_data = ImageList.from_df(df_test, path='.', cols="image",
                                    folder="images")
image_data.add_test(test_image_data)

# Tabular data
dep_var = 'species'
cont_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
procs = [FillMissing, Categorify, Normalize]
tab_data = (TabularList.from_df(df_train, path='.', cont_names=cont_names, procs=procs)
                           .split_by_idx(val_idx)
                           .label_from_df(cols=dep_var))
tab_data.add_test(TabularList.from_df(df_test, cont_names=cont_names,
                                      processor = tab_data.train.x.processor))

LabelLists;

Train: LabelList (108 items)
x: TabularList
sepal_length -1.4919; sepal_width 0.1212; petal_length -1.2816; petal_width -1.2870; ,sepal_length -0.6527; sepal_width 1.4527; petal_length -1.2816; petal_width -1.2870; ,sepal_length -1.3720; sepal_width 0.3431; petal_length -1.2246; petal_width -1.2870; ,sepal_length -0.8925; sepal_width 1.6746; petal_length -1.2246; petal_width -1.2870; ,sepal_length -0.5328; sepal_width 0.7870; petal_length -1.2816; petal_width -1.0272; 
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (27 items)
x: TabularList
sepal_length 1.6251; sepal_width 0.3431; petal_length 1.2848; petal_width 0.7914; ,sepal_length -1.4919; sepal_width 0.3431; petal_length -1.3387; petal_width -1.2870; ,sepal_length -0.4129; sepal_width -1.6541; petal_length 0.1442; petal_width 0.1419; ,sepal_length 0.5461; sepal_width 0.5651; petal_length 1.2848; petal_width 1.7007; ,sepal_length -0.1732; sepal_width -0.5445; petal_length 0.4293; petal_width 0.1419; 
y: CategoryL

Setelah itu data gambar dan data tabular digabungkan dengan menjadi fastai data bunch yang kemudian siap digunakan untuk training dan prediksi

In [6]:
integrate_train, integrate_valid, integrate_test = get_imagetabdatasets(image_data, tab_data)
db = DataBunch.create(integrate_train, integrate_valid, integrate_test,
                      path='.', bs=64)
#image normalization
db.norm, db.denorm = normalize_funcs_image_tab(*imagenet_stats)
db.add_tfm(db.norm)
x, y = next(iter(db.train_dl))

### Model

2 model deep learning yang digunakan untuk klasifikasi ini adalah resnet50 untuk data gambar dan model tabular untuk data tabular.

In [7]:
# cnn model for images, use Resnet50 as an example
cnn_arch = models.resnet50
cnn_out_sz = 256
# use fastai functions to get a cnn model
image_data_db = image_data.databunch()
image_data_db.c = cnn_out_sz
cnn_learn = cnn_learner(image_data_db, cnn_arch)
cnn_model = cnn_learn.model
# use fastai functions to get a tabular model
emb_szs = tab_data.train.get_emb_szs()
tab_out_sz = 8
tabular_model = TabularModel(emb_szs, len(cont_names), out_sz=tab_out_sz, layers=[8], ps=0.2)

Kemudian Kedua model tersebut digabungkan menggunakan CNNTabularModel dari library image_tabular

In [8]:
# get an integrated model that combines the two components and concatenate their outputs
integrate_model = CNNTabularModel(cnn_model,
                                  tabular_model,
                                  layers = [cnn_out_sz + tab_out_sz],
                                  ps=0.2,
                                  out_sz=3).to(device)

### Train

Model yang telah digabungkan diubah menjadi fastai learner untuk melakukan train pada model tersebut. Disini metrics yang digunakan adalah accuracy dan criterion yang digunakan adalah CrossEntropy loss yang menggabungkan nn.LogSoftmax() dan nn.NLLLoss().

In [9]:
# package everything in a fastai learner, add auc roc score as a metric
learn = Learner(db, integrate_model, metrics=accuracy, loss_func = nn.CrossEntropyLoss())

In [10]:
learn.layer_groups = [nn.Sequential(*flatten_model(cnn_learn.layer_groups[0])),
                      nn.Sequential(*flatten_model(cnn_learn.layer_groups[1])),
                      nn.Sequential(*(flatten_model(cnn_learn.layer_groups[2]) +
                                      flatten_model(integrate_model.tabular_model) +
                                      flatten_model(integrate_model.layers)))]

model dibagi menjadi group sehingga menggunakan metode freeze supaya model yang di update adalah model pada layer terakhir dan model pre-trained yang sudah memiliki weight yang baik tidak diupdate.

In [11]:
learn.freeze()
learn.fit(15, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.189409,1.215501,0.333333,00:20
1,1.052813,1.130703,0.333333,00:22
2,0.899223,1.111323,0.370370,00:21
3,0.808824,1.146767,0.481481,00:21
4,0.729435,1.155374,0.555556,00:21
5,0.693567,1.172577,0.592593,00:22
6,0.656075,1.159968,0.555556,00:21
7,0.634173,1.097355,0.555556,00:20
8,0.596451,0.990746,0.555556,00:21
9,0.573410,0.923385,0.555556,00:22


Setelah melakukan update pada layer terakhir maka melakukan training untuk semua model

In [13]:
learn.unfreeze()
learn.fit_one_cycle(20, max_lr=slice(1e-4,1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.185993,0.606983,0.777778,00:27
1,0.156521,0.534577,0.814815,00:27
2,0.122848,0.492743,0.777778,00:26
3,0.109265,0.497594,0.814815,00:26
4,0.094332,0.489232,0.814815,00:25
5,0.089765,0.378515,0.851852,00:25
6,0.104534,0.482902,0.851852,00:26
7,0.107817,0.767235,0.851852,00:26
8,0.104354,0.840637,0.851852,00:25
9,0.104167,0.808447,0.851852,00:25


In [14]:
preds, y = learn.get_preds(ds_type=DatasetType.Valid)
acc = accuracy(preds, y)
acc

tensor(0.8519)

### B. Analisa dari hasil train

Hasil dari training yang didapatkan, untuk model gabungan ini memiliki accuracy 86% yang merupakan hasil yang cukup baik. Akan tetapi, model ini memiliki kemungkinan overfitting yang dapat dilihat dengan adanya gap antara train loss dan valid loss. Apabila data ditambahkan maka akan memperkecil overfitting.

### C. Keunikan dari model

Model deep learning yang digunakan untuk klasifikasi 3 jenis bunga dengan menggabungkan data tabular dan data gambar adalah menggunakan model pre-trained cnn resnet-50 dan juga model tabular dari fast ai.

Resnet-50 adalah convolutional neural network yang memiliki 50 layers. Resnet sangat sering digunakan untuk menangani computer vision dimana ia menggunakan skip connection untuk menambah output dari layer sebelumnya ke layer setelahnya sehingga dapat membantu masalah vanishing gradient

Tidak hanya untuk menggabungkan penggunaan data tabular dan data gambar, Fastai juga merupakan library deep learning yang menyediakan model untuk vision, tabular, text, dan collaborative filtering. Disini saya menggunakan tabular model dimana variable continuous akan dimasukkan ke layer batch normalization dan variable kategori akan dimasukkan ke embedding layer yang kemudian akan digabungkan dan dimasukkan ke LinBnDrop sebelum ke layer linear yang akan menghasilkan output terakhir

Kemudian 2 model tersebut akan digabungkan menjadi 1 yang kemudian akan digabungkan dan ditambahkan fully connected layer sehingga dapat menghasilkan prediksi yang diinginkan.

Resource code: https://towardsdatascience.com/integrating-image-and-tabular-data-for-deep-learning-9281397c7318